# Data Mining

In this notebook, we'll query the PubChem database for results of a **SARS-CoV1 Papain-like protease** inhibitor assay (assay ID 1706).

We'll use the built in requests library to request data from the PubChem database using the [**PUG API**](https://pubchemdocs.ncbi.nlm.nih.gov/pug-rest). We'll encode our requests into a URL according to the [API documentation](https://pubchemdocs.ncbi.nlm.nih.gov/pug-rest), and then extract the data we want.

The base url is ```https://pubchem.ncbi.nlm.nih.gov/rest/pug```. In the cell below we add to that ```assay/aid/1706/record/CSV``` which specifies that we want all assay data from assay id (aid) 1706 in a ```.csv``` format.

In [2]:
import requests

r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/1706/record/CSV')
r.text.splitlines()

['Status: 400',
 'Code: PUGREST.BadRequest',
 'Message: Too many SIDs',
 'Detail: Assay record retrieval is limited to 10000 SIDs']

### That's an error

If the assay results contained fewer than 10000 datapoints, then we'd have our data and be just about done by now. An option is to use the [**PubChem FTP servers**](ftp://ftp.ncbi.nlm.nih.gov/pubchem) to get our data, but this is python club  so we're going to do it in code.

Whilst the dataset size is too big for downloading full ```csv``` files, we are allowed to download all compound IDs assosciated with the assay, so we'll download that as a text object.

I've added ```assay/aid/1706/cids/txt``` to the base url to specify compound IDs assosciated with that dataset in ```.txt``` format.

In [3]:
assay_cids = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/1706/cids/txt')
assay_cids = assay_cids.text.splitlines()

print(f'Number of compounds: {len(assay_cids)}')
print(assay_cids[0:10])

Number of compounds: 290726
['714726', '1259130', '2433099', '25237286', '135900541', '707587', '2598892', '6602856', '16190185', '280907']


## Batching
Instead of querying each compound individually, we can query batches (with a limited size), which is much faster. The batch size is determined by how many we can get away with without being blocked by the servers. I used a bit of trail and error to do this.


We'll test this out below by adding a query ```?cid=``` followed by a list of compound IDs (seperated by commas, no whitespace). In response, the servers will send us back the subset of the 1706 dataset containing those compound ids.

In [4]:
test_url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/1706/record/csv'
test_url += '?cid='
test_url += '748686,2529406,13185861'

test_request = requests.get(test_url)
print(test_request.text.splitlines())

['PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Inhibition', 'RESULT_TYPE,,,,,,,FLOAT', 'RESULT_DESCR,,,,,,,Normalized percent inhibition of the primary screen at a compound concentration of 6 micromolar.', 'RESULT_UNIT,,,,,,,PERCENT', 'RESULT_ATTR_CONC_MICROMOL,,,,,,,6', '89501,14745817,2529406,Inactive,1,,,1.14', '137708,22414508,13185861,Inactive,0,,,0.53', '166924,24805564,748686,Inactive,6,,,4.91']


The csv data we requested is returned in raw string form. An easy way to read this into a pandas DataFrame is to call the built in [**StringIO**](https://docs.python.org/2/library/stringio.html) function, which returns a file-like object from a string, which we can now treat as a normal csv file.

In [6]:
from io import StringIO
import pandas as pd

pd.read_csv(StringIO(test_request.text)).loc[:10,:]

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Inhibition
0,RESULT_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,FLOAT
1,RESULT_DESCR,NaN,NaN,NaN,NaN,NaN,NaN,Normalized percent inhibition of the primary s...
2,RESULT_UNIT,NaN,NaN,NaN,NaN,NaN,NaN,PERCENT
3,RESULT_ATTR_CONC_MICROMOL,NaN,NaN,NaN,NaN,NaN,NaN,6
4,89501,14745817.0,2529406.0,Inactive,1.0,NaN,NaN,1.14
5,137708,22414508.0,13185861.0,Inactive,0.0,NaN,NaN,0.53
6,166924,24805564.0,748686.0,Inactive,6.0,NaN,NaN,4.91


All empty values were replaced with ```NaN``` (not a number), which we'll have to handle later. After the 3rd row is the data I'm interested in: PubChemIDs and the Activity outcome (active or inactive according to some criteria). The data also contains an Inhibition metric in the form of a float, which may be useful.

To download the rest of the dataset I still need to do the following:
* Split the pubchem Compound IDs into manageable batches
* Make query urls from each batch
* Read the returning data into a pandas DataFrame and save it to disk

In [7]:
def MakeQueryURL(list_of_cids):
    l = ''.join([str(i)+',' for i in list_of_cids])

    query = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/1706/record/csv'
    query +='?cid=' + l
    return query
    

MakeQueryURL(assay_cids[0:10]) #test

'https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/1706/record/csv?cid=714726,1259130,2433099,25237286,135900541,707587,2598892,6602856,16190185,280907,'

In [8]:
def QueryRequestToDataFrame(url):
    r = requests.get(url)
    try:
        data = pd.read_csv(StringIO(r.text))
        # Clean the rows containing these values:
        data = data.loc[data['PUBCHEM_RESULT_TAG'] != 'RESULT_TYPE']
        data = data.loc[data['PUBCHEM_RESULT_TAG'] != 'RESULT_DESCR']
        data = data.loc[data['PUBCHEM_RESULT_TAG'] != 'RESULT_UNIT']
        data = data.loc[data['PUBCHEM_RESULT_TAG'] != 'RESULT_ATTR_CONC_MICROMOL']
        return data.reset_index(drop=True)
    except:
        print(r.status_code)
        print(r.text)
        # usually the problem is too many ids in the request

    

url = MakeQueryURL(assay_cids[0:1000])
QueryRequestToDataFrame(url) #test

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Inhibition
0,445,842574.0,644842.0,Inactive,6.0,NaN,NaN,4.94
1,660,842798.0,645071.0,Inactive,0.0,NaN,NaN,0.13
2,821,842962.0,645240.0,Inactive,0.0,NaN,NaN,-7.4
3,1611,843768.0,646070.0,Inactive,0.0,NaN,NaN,-0.4
4,1773,843933.0,646239.0,Inactive,3.0,NaN,NaN,2.74
...,...,...,...,...,...,...,...,...
995,289400,51085277.0,7205262.0,Inactive,0.0,NaN,NaN,0.71
996,289776,51086226.0,24891974.0,Inactive,0.0,NaN,NaN,-3.44
997,289940,51086521.0,135900541.0,Inactive,0.0,NaN,NaN,-2.83
998,289951,51086603.0,16807590.0,Inactive,5.0,NaN,NaN,4.79


## Ready to go!
We have:
* A list of compound IDs to use as queries to the AID1706 dataset
* A function that Makes query urls for this dataset
* A function that reads the response csv into a DataFrame, and cleans some rows that don't contain information I'm interested in

I'll divide the compund IDs into chunks of 1000 and save the output to disk. After that, I'll be able to loop through the rest of the chunks and append the csv on disk. The advantage of appending the file on disk on every iteration is that if the program fails a few minutes in, then I still have data saved. It also saves us from storing all of the data in RAM, which can be a limitation when working from a laptop.

In [9]:
chunk_size = 1000
chunks = [assay_cids[i:i+chunk_size] for i in range(0,len(assay_cids), chunk_size)]
url = MakeQueryURL(chunks[0])
data = QueryRequestToDataFrame(url)
data.to_csv('AID_1706.csv')
print(len(data))
data.head()

1000


,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Inhibition
0,445,842574.0,644842.0,Inactive,6.0,NaN,NaN,4.94
1,660,842798.0,645071.0,Inactive,0.0,NaN,NaN,0.13
2,821,842962.0,645240.0,Inactive,0.0,NaN,NaN,-7.4
3,1611,843768.0,646070.0,Inactive,0.0,NaN,NaN,-0.4
4,1773,843933.0,646239.0,Inactive,3.0,NaN,NaN,2.74


### Appending files
We can append data on to existing csv files with ```DataFrame.to_csv('filename.csv',mode='a')``` Where ```'a'``` means write in append mode, rather than overwriting the contents. We'll also use the option ```header = False``` to prevent ourselves from writing a new header every 1000 lines.


### Rate of requests

Unfortunately, parralelising this task is hard because the PubChem servers have request volume limits to prevent them from being overwhelmed. If we breach the limit, then we just won't get our data. Here are the PubChem guidelines:

* No more than 5 requests per second.
* No more than 400 requests per minute.
* No longer than 300 second running time per minute.

To slow down our request rate, I'll use the built-in function ```time.sleep(2)``` to pause the program for 2 seconds. We can also get the system time with ```time.time()``` to get an idea of how long each iteration takes, and use that to estimate when we'll be finished.

Usually about 20-25 minutes. We'll come back to it after it's done ☕

In [10]:
import time

for i,chunk_n in enumerate(chunks[1:]): # we already have the first chunk
    
    start_time = time.time()
    
    url_n = MakeQueryURL(chunk_n)
    df_n = QueryRequestToDataFrame(url_n)
    df_n.to_csv('AID_1706.csv',
                mode='a',
                header=False)
    
    time.sleep(2)
    
    finish_time = time.time()
    runtime = finish_time - start_time
    
    print(f'Chunk = {i} of {len(chunks)} \tTime/chunk = {round(runtime, 2)} s\
    \tETA = {round((runtime * (len(chunks)-i))/60)} minutes', end = '\r')

### Let's look at what we've made

In [13]:
Aid1706Data = pd.read_csv('AID_1706.csv', index_col=0)
print(len(Aid1706Data))
Aid1706Data.head()

290893


,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Inhibition
0,445,842574.0,644842.0,Inactive,6.0,NaN,NaN,4.94
1,660,842798.0,645071.0,Inactive,0.0,NaN,NaN,0.13
2,821,842962.0,645240.0,Inactive,0.0,NaN,NaN,-7.40
3,1611,843768.0,646070.0,Inactive,0.0,NaN,NaN,-0.40
4,1773,843933.0,646239.0,Inactive,3.0,NaN,NaN,2.74


### Are we Done?
Almost. To work with the data, I want the [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) code for each compound, which is a string that contains enough information to reconstruct the compound.

That means another round of requests to PubChem!

We'll also need to clean up the ```PUBCHEM_SID``` and ```PUBCHEM_CID``` columns, which have turned into ```floats``` (a result of the ```NaN``` filling on the intial requests).

In [14]:
Aid1706Data['PUBCHEM_SID'] = Aid1706Data['PUBCHEM_SID'].astype(int)
Aid1706Data['PUBCHEM_CID'] = Aid1706Data['PUBCHEM_CID'].astype(int)
Aid1706Data.head()

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Inhibition
0,445,842574,644842,Inactive,6.0,NaN,NaN,4.94
1,660,842798,645071,Inactive,0.0,NaN,NaN,0.13
2,821,842962,645240,Inactive,0.0,NaN,NaN,-7.40
3,1611,843768,646070,Inactive,0.0,NaN,NaN,-0.40
4,1773,843933,646239,Inactive,3.0,NaN,NaN,2.74


### Extra cleaning
There are a lot of ```NaN``` values in the columns ```PUBCHEM_ACTIVITY_URL``` and ```PUBCHEM_ASSAYDATA_COMMENT```, we'll check jsut how many, and then get rid of them.

In [15]:
for column in Aid1706Data:
    print(column, Aid1706Data[column].isna().sum())

PUBCHEM_RESULT_TAG 0
PUBCHEM_SID 0
PUBCHEM_CID 0
PUBCHEM_ACTIVITY_OUTCOME 0
PUBCHEM_ACTIVITY_SCORE 0
PUBCHEM_ACTIVITY_URL 290893
PUBCHEM_ASSAYDATA_COMMENT 290893
Inhibition 0


In [16]:
Aid1706Data.dropna(axis=1, inplace = True) # axis = 1 means drop the columns
Aid1706Data.head()

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,Inhibition
0,445,842574,644842,Inactive,6.0,4.94
1,660,842798,645071,Inactive,0.0,0.13
2,821,842962,645240,Inactive,0.0,-7.40
3,1611,843768,646070,Inactive,0.0,-0.40
4,1773,843933,646239,Inactive,3.0,2.74


## Getting the SMILES

In this section we're going to cross-reference our ```PUBCHEM_CID``` values with the PubChem database to get their SMILES codes, add that column to the ```Aid1706Data``` DataFrame and save it. Then we're done. 

### Query URL
I'm starting with this base URL again: ```https://pubchem.ncbi.nlm.nih.gov/rest/pug``` 
and add ```compound/cid/<put list of cids here>/property/CanonicalSMILES/CSV``` to access the batch of cmpounds, and return the canonical SMILES in ```csv``` format. When I ```print``` out some of the response string, we can see it's a ```csv``` with two columns.

In [23]:
l = ''.join([str(i)+',' for i in Aid1706Data['PUBCHEM_CID'][0:100]])
url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{l}/property/CanonicalSMILES/CSV'
r = requests.get(url)
data = r.text
print(data[0:500])

"CID","CanonicalSMILES"
644842,"COC(=O)C1=CC=CC=C1NC(=O)C2CC(=O)N(C2)C3=CC=C(C=C3)F"
645071,"CCOC(=O)CNC(=O)CSC1=NN=NN1C2=C(C=CC(=C2)C)C"
645240,"C1CN(CCC1(C2=CN=CC=C2)O)C(=O)C3=CC=CC=C3F"
646070,"CC12CC3(CN(C1)C(N(C2)C3)C4=CC(=C(C=C4)O)OC)C"
646239,"C1CC(OC1)CN(CC2=CC3=CC4=C(C=C3NC2=O)OCCO4)S(=O)(=O)C5=CC=CC=C5"
646359,"CC1=NN=C(S1)SC(C)C(=O)NC2=CC=C(C=C2)S(=O)(=O)N3CCCCCC3"
646468,"COCCN1C(=NN=C1SCC2=CN=CC=C2)C3=CC=CO3"
647129,"CCN1C(=NN=C1SCC(=O)NC2=CC=CC(=C2)C(=O)OCC)C3=C(OC=C3)C"
6603025,"C


We'll have to use ```StringIO``` again to read the data into a ```DataFrame```. I'll split the ```Aid1706Data['PUBCHEM_CID']``` column into chunks and make a function to generate requests and put the output into a ```DataFrame```.

In [27]:
chunk_size = 450
chunks = [Aid1706Data['PUBCHEM_CID'][i:i+chunk_size] for i in range(0,len(Aid1706Data['PUBCHEM_CID']), chunk_size)]

def CIDtoSmiles(cid_list):
    l = ''.join([str(i)+',' for i in cid_list])
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{l}/property/CanonicalSMILES/CSV'
    r = requests.get(url)
    data = r.text
    return pd.read_csv(StringIO(data))

CIDtoSmiles(chunks[0]).head()

,CID,CanonicalSMILES
0,644842,COC(=O)C1=CC=CC=C1NC(=O)C2CC(=O)N(C2)C3=CC=C(C...
1,645071,CCOC(=O)CNC(=O)CSC1=NN=NN1C2=C(C=CC(=C2)C)C
2,645240,C1CN(CCC1(C2=CN=CC=C2)O)C(=O)C3=CC=CC=C3F
3,646070,CC12CC3(CN(C1)C(N(C2)C3)C4=CC(=C(C=C4)O)OC)C
4,646239,C1CC(OC1)CN(CC2=CC3=CC4=C(C=C3NC2=O)OCCO4)S(=O...


Now we're ready to process all of the compounds. We'll save as we go and use a progress bar from [**tqdm**](https://github.com/tqdm/tqdm). The standard ```tqdm``` syntax is:
```python
from tqdm import tqdm
for i in tqdm(range(10000)):
    ...
    ```
```76%|████████████████████████        | 7568/10000 [00:33<00:10, 229.00it/s]```

which is a nice sanity check on slow jobs. We'll use a notebook-specific progress bar: ```tqdm_notebook``` here. The job takes > 10 minutes so time for another ☕

In [28]:
from tqdm.notebook import tqdm_notebook # progress bar!

chunk_size = 450
chunks = [Aid1706Data['PUBCHEM_CID'][i:i+chunk_size] for i in range(0,
                                                                    len(Aid1706Data['PUBCHEM_CID']), 
                                                                    chunk_size)]

#save the first chunk
smiles_cidMapping = CIDtoSmiles(chunks[0]).to_csv('SmilesCIDMapping.csv')

for i in tqdm_notebook(chunks[1:]):
    # should be in same order as in the dataframe
    data_retreival = CIDtoSmiles(i)
    data_retreival.to_csv('SmilesCIDMapping.csv', mode = 'a', header = None)
    time.sleep(0.1)
    
smiles_cidMapping = pd.read_csv('SmilesCIDMapping.csv', index_col = 0)
smiles_cidMapping.head()

,CID,CanonicalSMILES
0,644842,COC(=O)C1=CC=CC=C1NC(=O)C2CC(=O)N(C2)C3=CC=C(C...
1,645071,CCOC(=O)CNC(=O)CSC1=NN=NN1C2=C(C=CC(=C2)C)C
2,645240,C1CN(CCC1(C2=CN=CC=C2)O)C(=O)C3=CC=CC=C3F
3,646070,CC12CC3(CN(C1)C(N(C2)C3)C4=CC(=C(C=C4)O)OC)C
4,646239,C1CC(OC1)CN(CC2=CC3=CC4=C(C=C3NC2=O)OCCO4)S(=O...


In [3]:
import pandas as pd # came back to this 

Aid1706Data = pd.read_csv('AID_1706.csv', index_col=0)

smiles = pd.read_csv('SmilesCIDMapping.csv', index_col=1) #make the CIDs the index
smiles = smiles.drop('Unnamed: 0', axis=1) 
Aid1706Data['smiles'] = smiles.loc[Aid1706Data['PUBCHEM_CID']].reset_index(drop=True)
Aid1706Data.to_csv('AID_1706.csv')

Aid1706Data.head()

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,Inhibition,smiles
0,445,842574,644842,Inactive,6.0,4.94,COC(=O)C1=CC=CC=C1NC(=O)C2CC(=O)N(C2)C3=CC=C(C...
1,660,842798,645071,Inactive,0.0,0.13,CCOC(=O)CNC(=O)CSC1=NN=NN1C2=C(C=CC(=C2)C)C
2,821,842962,645240,Inactive,0.0,-7.40,C1CN(CCC1(C2=CN=CC=C2)O)C(=O)C3=CC=CC=C3F
3,1611,843768,646070,Inactive,0.0,-0.40,CC12CC3(CN(C1)C(N(C2)C3)C4=CC(=C(C=C4)O)OC)C
4,1773,843933,646239,Inactive,3.0,2.74,C1CC(OC1)CN(CC2=CC3=CC4=C(C=C3NC2=O)OCCO4)S(=O...


## Now we're done
Time for another ☕